In [3]:
import pandas as pd
import sklearn as sk
import numpy as np
from sklearn.decomposition import LatentDirichletAllocation

datadir = '../processed_data/'

### Read in dummy Data

In [4]:
data = pd.read_csv(datadir + 'database.csv')

In [5]:
data.columns

Index(['Unnamed: 0', 'account_id', 'trans_id', 'category', 'payee', 'k-symbol',
       'date', 'amount', 'mean_income', 'Initial Balance'],
      dtype='object')

In [6]:
cx = pd.read_csv(datadir + 'customers.csv')

In [7]:
print([len(set(cx.account_id)), len(set(data.payee))])
print(max(data.payee))

[4500, 1404]
1403


In [8]:
# Remove inc
#data = data[data.amount > 0]
# data = data.sort_values(['account_id'])
# acct_lkp = {'account_id': data.account_id, 'id': range(cx.shape[0])}
#data

### Data manipulation

In [15]:
# Use Implementation from David Blei's lab, since Scikit-learn version is a little odd.
from bleilab import onlineldavb

In [ ]:
olda = onlineldavb.OnlineLDA(vocab, K, D, 1./K, 1./K, 1024., 0.7)

In [12]:

# [for each customer, create a row detailing counts for each transaction type]
# e.g.
#
# ID   Type1 Type2 Type3 Type4 Type5
# ====================================
# 001    0     1     0     3     1
# 002    0     0     1     1     0
# 003    1     0     0     2     0
# ...   ..    ..    ..    ..    ..

tfData = np.zeros([cx.shape[0]+1, max(data.payee)+1])
cxTot  = data.shape[0]
cxNum  = 0
for ii in range(data.shape[0]-1):
    tfData[cxNum, data.payee[ii]] += 1
    cxNum += data.account_id[ii+1] != data.account_id[ii]

# do final customer
# cII = ii;
# for ii in range(cII, data.shape[0]-1):
#     tfData[cxNum, data.payee[ii]] += 1
tfData

# Apologies, this is SLOW and inelegant: only used once in hackathon

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [20]:
tfData.shape

238.0

### Test scikit-learn LDA
(Turns out that this is **much** faster than R's current package implementation (not sure if R uses stochastic/minibatch variational EM))

In [12]:
# Inspect example dataset to understand type of 
# ===> CHECK 
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=1000, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()
tf
# ===> IE. want sparse matrix where every row is a customer, every column is a transaction type.

In [17]:
lda = LatentDirichletAllocation(n_topics=25, 
                                max_iter=5, learning_method='online', 
                                learning_offset=50.,random_state=0, verbose=True, n_jobs=-1).fit(tfData)

[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    0.1s finished
[Parallel(n_jobs=4)]

In [19]:
# files were deleted when server taken down
# We only have indices now.
if False:
    file1 = open(datadir + "payees.txt", "r")
    payees = file1.readlines()
    file1.close()
    file2 = open(datadir + "types.txt", "r")
    types = file2.readlines()
    file2.close()

FileNotFoundError: [Errno 2] No such file or directory: '../processed_data/payees.txt'

In [27]:
num_top_trans = 10
for topic_idx, topic in enumerate(lda.components_):
    print("Topic %d:" % (topic_idx))
    print(" ".join([str(i)
                    for i in topic.argsort()[:-num_top_trans - 1:-1]]))

Topic 0:
1160 280 840 897 410 980 794 263 385 519
Topic 1:
334 314 1071 396 372 1086 1233 1139 1149 648
Topic 2:
1249 1291 1001 856 582 355 701 363 482 899
Topic 3:
107 311 989 1067 537 1028 1115 1323 269 38
Topic 4:
786 383 375 1109 1134 363 895 430 806 1175
Topic 5:
160 892 823 579 834 967 1047 265 587 845
Topic 6:
52 90 86 1158 954 913 899 916 49 51
Topic 7:
52 90 916 899 449 1054 913 1158 86 1128
Topic 8:
948 90 449 1119 413 916 414 1351 899 126
Topic 9:
258 456 1029 810 407 662 205 256 498 54
Topic 10:
1372 526 204 1033 1335 773 711 776 1109 185
Topic 11:
449 90 1265 52 560 413 916 899 913 911
Topic 12:
90 52 449 325 1136 86 913 899 916 1158
Topic 13:
691 90 899 733 52 911 913 1025 1332 1128
Topic 14:
90 899 52 913 651 1158 318 986 49 200
Topic 15:
90 548 52 86 899 916 913 911 1158 954
Topic 16:
90 449 913 52 899 86 49 1128 399 361
Topic 17:
101 1107 440 1335 1381 406 117 1324 103 1386
Topic 18:
86 52 90 548 954 899 51 1073 49 916
Topic 19:
90 52 275 86 899 1073 913 1158 916 954
T

##### Problems with scikit package:
It is not at all obvious how to extract the categorical distributions by customer from the python LDA implementation: this is a shame as it is extremely fast. Perhaps this can be explored later - Matt Hoffman's script upon which the sklean script is based is clearer, but written in Python 2.x, which I don't currently have installed. 

### Update: Fixed! (AB)

In [40]:
# Not straight-forward to obtain the distribution over documents from the LDA object.
# Having gone through the code, and Matt Hoffman's paper, it seems this is what 
# we want (the e-step returns the *parameters* of the Dirichlet distribution. Mean of
# dirichlet requires only normalisation thereof).

doc_topics_distr, _ = lda._e_step(tfData, cal_sstats=False, random_init=False)
sum_gammas          = doc_topics_distr.sum(axis = 1)
cxDistns            = doc_topics_distr / sum_gammas[:, np.newaxis]

typeClusters        = topic

[Parallel(n_jobs=4)]: Done   4 out of   4 | elapsed:    1.4s finished


In [41]:
print(cxDistns[0:5,:])

[[  1.67364017e-04   1.67364017e-04   1.67364017e-04   1.67364017e-04
    1.67364017e-04   1.67364017e-04   9.81306920e-01   1.67364017e-04
    1.67364017e-04   1.67364017e-04   1.67364017e-04   1.67364017e-04
    1.48437078e-02   1.67364017e-04   1.67364017e-04   1.67364017e-04
    1.67364017e-04   1.67364017e-04   1.67364017e-04   1.67364017e-04
    1.67364017e-04   1.67364017e-04   1.67364017e-04   1.67364017e-04
    1.67364017e-04]
 [  8.36820084e-05   8.36820084e-05   8.36820084e-05   8.36820084e-05
    8.36820084e-05   8.36820084e-05   2.02201608e-01   8.36820084e-05
    8.36820084e-05   8.36820084e-05   8.36820084e-05   6.87031965e-01
    9.67446136e-02   8.36820084e-05   8.36820084e-05   1.22644917e-02
    8.36820084e-05   8.36820084e-05   8.36820084e-05   8.36820084e-05
    8.36820084e-05   8.36820084e-05   8.36820084e-05   8.36820084e-05
    8.36820084e-05]
 [  3.41880342e-04   3.41880342e-04   3.41880342e-04   3.41880342e-04
    3.41880342e-04   3.41880342e-04   3.52449677e-

### Instead interfact with R's version
(Unfortunately this is extremely slow.)

### Update: No longer needed (see above)

In [23]:
# R scripts may be called from python via: 
from subprocess import call
call (["/usr/bin/Rscript", "--vanilla", "./r-lda.r"])
    # see script r-lda.R

FileNotFoundError: [Errno 2] No such file or directory: '/usr/bin/Rscript'

In [ ]:
# load inputs in from R
cxDistns     = pd.read_csv('lda_distns.csv')
typeClusters = pd.read_csv('lda_clusters.csv')